In [7]:
import json

# --- Scoring Constants based on your JSON notes ---
SCORE_IDEAL = 1.0
SCORE_MILD = 0.7
SCORE_MODERATE = 0.4
SCORE_SEVERE = 0.0

# --- Deviation Thresholds (as a percentage of the ideal range width) ---
# These are tunable parameters to define what counts as a mild, moderate, or severe deviation.
MILD_DEVIATION_THRESHOLD = 0.25  # 25%
MODERATE_DEVIATION_THRESHOLD = 0.50 # 50%

def get_parameter_safety_score(predicted_val, ideal_range):
    """
    Calculates a safety score (1.0, 0.7, 0.4, 0.0) for a single parameter
    by comparing the predicted value to the ideal range.
    """
    if predicted_val is None:
        return None # Ignore null values

    min_ideal, max_ideal = ideal_range
    
    # Check if the value is within the ideal range
    if min_ideal <= predicted_val <= max_ideal:
        return SCORE_IDEAL

    # Calculate deviation if outside the range
    range_width = max_ideal - min_ideal
    if range_width == 0: # Avoid division by zero for fixed-point ideals
        return SCORE_SEVERE if predicted_val != min_ideal else SCORE_IDEAL

    if predicted_val > max_ideal:
        deviation = predicted_val - max_ideal
    else: # predicted_val < min_ideal
        deviation = min_ideal - predicted_val
        
    deviation_ratio = deviation / range_width

    # Assign score based on deviation severity
    if deviation_ratio <= MILD_DEVIATION_THRESHOLD:
        return SCORE_MILD
    elif deviation_ratio <= MODERATE_DEVIATION_THRESHOLD:
        return SCORE_MODERATE
    else:
        return SCORE_SEVERE

def analyze_crop_data(data):
    """
    Analyzes the full crop data JSON to calculate stage-wise and overall health scores.
    """
    print(f"Analyzing Health Score for {data['crop']} in {data['region']}\n")

    feature_weights = data['scoring']['feature_weights']
    
    # A mapping to group individual parameters into the weighted feature categories
    param_to_feature_map = {
        'tmin_c': 'temperature', 'tmax_c': 'temperature',
        'rain_mm': 'rain',
        'rh_pct': 'humidity_vpd', 'vpd_kpa': 'humidity_vpd',
        'wind_kmph': 'wind',
        'solar_wm2': 'radiation_et0', 'et0_mm': 'radiation_et0',
        'soil_moisture_pct': 'soil', 'soil_temp_c': 'soil',
        'leaf_wetness_hours': 'leaf_wetness',
        'uv_index': 'uv'
    }

    stage_scores = []
    total_importance_weight = 0
    
    # --- Loop 1: Calculate the safety score for each stage ---
    for stage in data['stages']:
        stage_name = stage['name']
        importance_weight = stage['importance_weight']
        total_importance_weight += importance_weight
        
        # This will hold the scores for each feature category (e.g., temperature, rain)
        feature_scores = {key: [] for key in feature_weights.keys()}

        # Iterate through each parameter in the 'ideal' definition
        for param, ideal_range in stage['ideal'].items():
            predicted_val = stage['predicted'].get(param)
            
            # Calculate the safety score for this specific parameter
            param_score = get_parameter_safety_score(predicted_val, ideal_range)

            if param_score is not None:
                # Find which feature category this parameter belongs to
                feature_category = param_to_feature_map.get(param)
                if feature_category:
                    feature_scores[feature_category].append(param_score)

        # Calculate the weighted average safety score for the stage
        stage_safety_numerator = 0
        stage_safety_denominator = 0

        for feature, scores in feature_scores.items():
            if scores: # Only consider features that had valid data
                avg_score_for_feature = sum(scores) / len(scores)
                weight = feature_weights[feature]
                stage_safety_numerator += avg_score_for_feature * weight
                stage_safety_denominator += weight
        
        stage_safety_score = (stage_safety_numerator / stage_safety_denominator) if stage_safety_denominator > 0 else 0
        
        stage_scores.append({
            'name': stage_name,
            'safety_score': stage_safety_score,
            'importance_weight': importance_weight
        })

    # --- Loop 2: Calculate the final overall score using the aggregation formula ---
    overall_score_numerator = 0
    for stage_info in stage_scores:
        overall_score_numerator += stage_info['safety_score'] * stage_info['importance_weight']
        print(f"Stage: {stage_info['name']:<25} | Stage Safety Score: {stage_info['safety_score']:.2f}")

    overall_health_score = 100 * (overall_score_numerator / total_importance_weight)

    print("\n")
    print(f"OVERALL CROP HEALTH SCORE: {overall_health_score:.2f} / 100")
  

In [8]:
# --- Main execution ---
# Load the JSON file
try:
    with open('wheat_time_line_data.json', 'r') as f:
        crop_data = json.load(f)
    
    # Run the analysi
    # s
    analyze_crop_data(crop_data)

except FileNotFoundError:
    print("Error: 'wheat_time_line_data.json' not found. Please ensure the file is in the same directory.")
except json.JSONDecodeError:
    print("Error: Could not decode the JSON file. Please check its format.")

Analyzing Health Score for Wheat in Central India

Stage: Sowing & Germination      | Stage Safety Score: 0.94
Stage: Seedling / Emergence      | Stage Safety Score: 0.94
Stage: Tillering                 | Stage Safety Score: 0.94
Stage: Stem Elongation           | Stage Safety Score: 0.94
Stage: Booting                   | Stage Safety Score: 0.94
Stage: Heading                   | Stage Safety Score: 0.94
Stage: Flowering / Anthesis      | Stage Safety Score: 0.94
Stage: Grain Filling (Milk)      | Stage Safety Score: 0.88
Stage: Grain Filling (Dough)     | Stage Safety Score: 0.94
Stage: Ripening                  | Stage Safety Score: 1.00
Stage: Harvest                   | Stage Safety Score: 1.00


OVERALL CROP HEALTH SCORE: 94.04 / 100
